# TA後形状の予測

##### 作成日：　2024.12.9
##### データ元：　工程内形状データ、ライン作業実績
##### 解析手順：　データ抽出・前処理　⇒機械学習（ランダムフォレスト）⇒予測　⇒グラフ化

## 1. データ抽出

In [1]:
import sqlite3
import pandas as pd

# --------------------------------------------------------------
# 抽出対象
# --------------------------------------------------------------
user_code = "D250"
thickness = 30.0
width = 495


# --------------------------------------------------------------
# データベース接続＆抽出
# --------------------------------------------------------------
dbname = "F:\\金属箔事業部\\秘L\\旧箔\\箔\\品質保証Gr\\QC150_品質データ\\010_オフライン形状測定装置データ_工場\\新形状測定装置\\★SQLite取り込みプログラム\\DB\\keijyo_new.db"
#dbname = "./DB/keijyo_new.db"

# CASE WHEN THEN: 文字列の置換
# MAX : GROUP BY条件のなかで、m_timeの最大値を抽出

with sqlite3.connect(dbname) as conn1:
    sql = f'''
            SELECT
                * ,
                MAX(m_time),
                CASE 
                    WHEN position = 'f' THEN 'F'
                    WHEN position = 'Ｆ' THEN 'F'
                    WHEN position = 'b' THEN 'B'
                    WHEN position = 'Ｂ' THEN 'B'
                    ELSE position
                END AS new_position
            FROM tbl_01
            WHERE
                (customer = "{user_code}")
                AND (thickness = "{thickness}")
                AND (width = "{width}")
                AND (treat_code = "4" OR treat_code = "9")
            GROUP BY
                coil_num,
                coildiv_num,
                ope_code,
                treat_code,
                CASE 
                    WHEN position = 'f' THEN 'F'
                    WHEN position = 'Ｆ' THEN 'F'
                    WHEN position = 'b' THEN 'B'
                    WHEN position = 'Ｂ' THEN 'B'
                    ELSE position
                END
            ORDER BY m_date ASC
        '''
    df = pd.read_sql(sql, con=conn1)


# --------------------------------------------------------------
# DF作成
# --------------------------------------------------------------

# コイル番号
df["new_coil"] = df["coil_num"] + "-"+ df["coildiv_num"]

# 処理コードと位置の新規列　["9F","9B","4F","4B"] = ["FCR_F","FCR_B","TA_F","TA_B"]
df["process_position"] = df["treat_code"] + df["new_position"]

# 使用する列の指定
new_col = ["m_date", "new_coil", "process_position"] + [f"i{i}" for i in range(1, 21)]
df = df[new_col]

# CSV出力
#df.to_csv("data.csv", encoding='shift-jis')

df.head(3)

,m_date,new_coil,process_position,i1,i2,i3,i4,i5,i6,i7,...,i11,i12,i13,i14,i15,i16,i17,i18,i19,i20
0,2023-04-15 00:00:00,49134-11,9B,18.136928,3.790268,0.0,0.558058,0.992577,2.033761,2.587749,...,0.333904,0.013008,0.307649,0.652011,0.720563,1.595945,2.631035,4.617211,10.478505,17.890834
1,2023-04-15 00:00:00,49134-11,9F,14.397729,3.015333,0.0,0.479469,0.775821,1.638432,2.051087,...,0.772083,0.248304,0.395933,0.899396,0.736544,0.840219,1.414581,3.619171,8.066631,13.722709
2,2023-04-15 00:00:00,49134-12,9B,12.868052,4.347836,0.0,0.476016,1.263866,1.913309,2.823925,...,0.959822,0.789794,1.568435,1.387556,0.618093,0.651241,1.425827,2.963362,6.921700,10.121797


In [2]:
df_MF = df[df["process_position"] == "9F"]    # FCR F
df_MB = df[df["process_position"] == "9B"]    # FCR B
df_TF = df[df["process_position"] == "4F"]    # TA F
df_TB = df[df["process_position"] == "4B"]    # TA B

df_MF.columns = ["date_FCR_F", "new_coil", "pp_FCR_F"] + [f"FCR_F_{i}" for i in range(1, 21)]
df_MB.columns = ["date_FCR_B", "new_coil", "pp_FCR_B"] + [f"FCR_B_{i}" for i in range(1, 21)]
df_TF.columns = ["date_TA_F", "new_coil", "pp_TA_F"] + [f"TA_F_{i}" for i in range(1, 21)]
df_TB.columns = ["date_TA_B", "new_coil", "pp_TA_B"] + [f"TA_B_{i}" for i in range(1, 21)]

# "new_coil"をキーに各DFを結合
df_a1 = pd.merge(df_MF, df_MB, on="new_coil", how='left')
df_a1 = pd.merge(df_a1, df_TF, on="new_coil", how='left')
df_a1 = pd.merge(df_a1, df_TB, on="new_coil", how='left')

#df2.to_csv("data2.csv", encoding='shift-jis')

df_a1.head(3)

,date_FCR_F,new_coil,pp_FCR_F,FCR_F_1,FCR_F_2,FCR_F_3,FCR_F_4,FCR_F_5,FCR_F_6,FCR_F_7,...,TA_B_11,TA_B_12,TA_B_13,TA_B_14,TA_B_15,TA_B_16,TA_B_17,TA_B_18,TA_B_19,TA_B_20
0,2023-04-15 00:00:00,49134-11,9F,14.397729,3.015333,0.000000,0.479469,0.775821,1.638432,2.051087,...,2.006524,0.570906,0.501261,0.575592,0.108840,0.000000,1.169751,2.753668,7.081118,13.974039
1,2023-04-15 00:00:00,49134-12,9F,15.557247,4.253771,0.108134,0.819745,1.503837,1.987190,2.420420,...,1.252476,0.168589,0.007054,0.076176,0.000000,0.433399,1.830434,2.864033,6.868363,11.273258
2,2023-05-07 00:00:00,49135-1,9F,14.571781,5.226943,1.243542,1.725209,0.962206,0.626832,0.069162,...,2.305909,1.347314,0.940886,0.421182,0.235367,0.169316,1.139534,2.968099,7.583181,15.721907


### 熱処理条件
##### ライン作業実績のCSV読み込み

In [3]:
import cx_Oracle


# -----------------------------------------------------------------
# サーバー接続設定
# -----------------------------------------------------------------
SvrName = "WMDBMH21"
Port  = 1521
SvsName = "MSGK"
USRName = "HAKU1"
PWD = "HAKU1"

# ----------------------------------------------------------------------
# 日報表示　データ前処理関数
# ----------------------------------------------------------------------
def make_df(opt_date, customer, thickness):          # 引数： opt_date：作業年月日時分

    data_list = []
    
    # 箔システム(MSGK)からデータ抽出
    try:
        # OracleDBとの接続
        tns = cx_Oracle.makedsn(SvrName, Port, SvsName)
        conOcl = cx_Oracle.connect(USRName, PWD, tns)

        curOcl = conOcl.cursor()
        curOcl.arraysize = 1000

        # ライン作業実績TBLから期間が一致する項目を読み込み
        sql_Ocl = f'''
            SELECT
                HAKU.ライン作業実績_V.協定仕様番号_需要家コード,
                HAKU.ライン作業実績_V.協定仕様番号_品種コード,
                HAKU.ライン作業実績_V.協定仕様番号_鋼種コード,
                HAKU.ライン作業実績_V.協定仕様番号_調質コード,
                HAKU.ライン作業実績_V.協定仕様番号_仕上コード,
                HAKU.ライン作業実績_V.協定仕様番号_公称板厚,
                HAKU.ライン作業実績_V.協定仕様番号_公称板幅,
                HAKU.ライン作業実績_V.協定仕様番号_連番,
                HAKU.ライン作業実績_V.協定仕様番号_改訂番号,
                HAKU.ライン作業実績_V.コイル番号,
                HAKU.ライン作業実績_V.コイル分割番号,
                HAKU.ライン作業実績_V.作業年月日時分,
                HAKU.ライン作業実績_V.工程コード_作業コード,
                HAKU.ライン作業実績_V.工程コード_ラインコード1,
                HAKU.ライン作業実績_V.工程コード_ラインコード2,
                HAKU.ライン作業実績_V.工程コード_処理種別,
                HAKU.ライン作業実績_V.前面出し区分,
                HAKU.ライン作業実績_V.作業後コイル長さ,
                HAKU.ライン作業実績_工程個別_V.熱処理_速度,
                HAKU.ライン作業実績_工程個別_V.熱処理_張力入側,
                HAKU.ライン作業実績_工程個別_V.熱処理_張力中央,
                HAKU.ライン作業実績_工程個別_V.熱処理_張力出側,
                HAKU.ライン作業実績_工程個別_V.熱処理_H2流量,
                HAKU.ライン作業実績_工程個別_V.熱処理_炉温1,
                HAKU.ライン作業実績_工程個別_V.熱処理_炉温2,
                HAKU.ライン作業実績_工程個別_V.熱処理_炉温3,
                HAKU.ライン作業実績_工程個別_V.熱処理_炉温4,
                HAKU.ライン作業実績_工程個別_V.熱処理_炉温5,
                HAKU.ライン作業実績_工程個別_V.熱処理_炉温6,
                HAKU.ライン作業実績_工程個別_V.熱処理_炉温7,
                HAKU.ライン作業実績_工程個別_V.熱処理_炉温8,
                HAKU.ライン作業実績_工程個別_V.熱処理_炉温9,
                HAKU.ライン作業実績_工程個別_V.熱処理_炉温10,
                HAKU.ライン作業実績_工程個別_V.熱処理_炉温11,
                HAKU.ライン作業実績_工程個別_V.熱処理_炉温12,
                HAKU.ライン作業実績_工程個別_V.熱処理_炉温13,
                HAKU.ライン作業実績_工程個別_V.熱処理_炉温14,
                HAKU.ライン作業実績_工程個別_V.熱処理_炉温15,
                HAKU.ライン作業実績_工程個別_V.熱処理_炉温16 
            FROM 
                HAKU.ライン作業実績_V 
                LEFT JOIN HAKU.ライン作業実績_工程個別_V 
                ON (HAKU.ライン作業実績_V.コイル番号 = HAKU.ライン作業実績_工程個別_V.コイル番号) 
                AND (HAKU.ライン作業実績_V.コイル分割番号 = HAKU.ライン作業実績_工程個別_V.コイル分割番号)
                AND (HAKU.ライン作業実績_V.工程順 = HAKU.ライン作業実績_工程個別_V.工程順) 
                AND (HAKU.ライン作業実績_V.工程順子番 = HAKU.ライン作業実績_工程個別_V.工程順子番)
            WHERE
                (HAKU.ライン作業実績_V.作業年月日時分 >= '{opt_date}')
                AND (HAKU.ライン作業実績_V.前面出し区分 = '0')
                AND (HAKU.ライン作業実績_V.工程コード_作業コード = '2')
                AND (HAKU.ライン作業実績_V.工程コード_処理種別 = '4')
                AND (HAKU.ライン作業実績_V.協定仕様番号_需要家コード = '{customer}')
                AND (HAKU.ライン作業実績_V.協定仕様番号_公称板厚 = '{thickness}')
            ORDER BY
                ライン作業実績_V.作業年月日時分
        '''

        # カーソルを実行
        curOcl.execute(sql_Ocl)

        # リストとして全件取得し、データフレームに変換
        data_list = curOcl.fetchall()

    # Oracleエラー
    except cx_Oracle.OperationalError as e:
        error, = e.args
        print("Oracle関連のエラーが発生しました。")
        print(f"app4: Oracle ErrorCode: {error.code}")
        print(f"app4: Oracle ErrorMessage: {error.message}")

    # 一般エラー
    except Exception as e:
        # エラーメッセージの表示
        print(f"app4: Error Occurred: {e}")

    finally:
        #　DB接続の終了
        if conOcl is not None:
            curOcl.close()
            conOcl.close()


    df_ocl = pd.DataFrame(data_list)
    
    df_ocl.columns = ["需要家コード",
                      "品種コード",
                      "鋼種コード",
                      "調質コード",
                      "仕上コード",
                      "公称板厚",
                      "公称板幅",
                      "連番",
                      "改訂番号",
                      "コイル番号",
                      "コイル分割番号",
                      "作業年月日時分",
                      "作業コード",
                      "ラインコード1",
                      "ラインコード2",
                      "処理種別",
                      "前面出し区分",
                      "作業後コイル長さ",
                      "熱処理速度",
                      "張力入側",
                      "張力中央",
                      "張力出側",
                      "H2流量",             
                ] + [f"炉温{i}" for i in range(1, 17)]
    
    return df_ocl


df_b1 = make_df("202304010000", "D250", "0300")

# コイル番号
df_b1["new_coil"] = df_b1["コイル番号"] + "-"+ df_b1["コイル分割番号"]

# 使用する列の指定
new_col = ["作業年月日時分", "new_coil", "熱処理速度"] + [f"炉温{i}" for i in range(1, 17)]
df_b1 = df_b1[new_col]

df_b1.sample(3)

,作業年月日時分,new_coil,熱処理速度,炉温1,炉温2,炉温3,炉温4,炉温5,炉温6,炉温7,炉温8,炉温9,炉温10,炉温11,炉温12,炉温13,炉温14,炉温15,炉温16
125,202409142145,53555-1,40.0,657,654,656,654,656,655,649,656,512,403,298,301,208,194,98,29
117,202408210055,53302-22,52.5,656,655,657,654,655,655,650,656,516,403,305,301,206,194,106,31
100,202406220323,53166-2,52.5,658,655,656,654,654,655,651,656,515,419,301,301,212,189,94,26


In [4]:
# "new_coil"をキーに各DFを結合
df_c1 = pd.merge(df_a1, df_b1, on="new_coil", how='left')

df_c1.to_csv("data0.csv", encoding='shift-jis')

df_c1.sample(3)

,date_FCR_F,new_coil,pp_FCR_F,FCR_F_1,FCR_F_2,FCR_F_3,FCR_F_4,FCR_F_5,FCR_F_6,FCR_F_7,...,炉温7,炉温8,炉温9,炉温10,炉温11,炉温12,炉温13,炉温14,炉温15,炉温16
112,2024-08-28 00:00:00,53474-1,9F,16.790717,3.791754,0.000000,0.358386,0.822472,2.051964,1.281495,...,649.0,656.0,511.0,403.0,297.0,301.0,204.0,204.0,95.0,29.0
99,2024-07-15 00:00:00,53298-2,9F,15.669783,5.561162,1.549341,0.803334,0.204147,0.522187,0.000000,...,650.0,656.0,516.0,414.0,303.0,301.0,210.0,194.0,109.0,32.0
125,2024-09-17 00:00:00,53577-22,9F,12.368234,2.039884,0.000000,0.354500,0.565953,1.347758,1.795442,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
